In [1]:
!pip install gradientai --upgrade

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.7/410.7 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.6/137.6 kB 7.9 MB/s eta 0:00:00


In [9]:
import os
os.environ['GRADIENT_ACCESS_TOKEN'] = ""
os.environ['GRADIENT_WORKSPACE_ID'] = ""

In [10]:
# for parsing csv source file
import csv

# gradient library
from gradientai import Gradient

path_to_csv_file = "./deathnote1.csv"

In [11]:
# find pairs of lines where someone else speaks, then Rick speaks
# append them as two separate lines in rows_to_keep
print("Parsing data...")
rows_to_keep = []
with open(path_to_csv_file, encoding="utf-8-sig") as f:
  reader = csv.DictReader(f, delimiter=",")
  last_row = None
  for row in reader:
    if "Light" == row["name"] and last_row is not None:
      rows_to_keep.append(last_row)
      rows_to_keep.append(row)
      last_row = None
    else:
      last_row = row

# create a role-playing prompt for training and
# later for prompting
role_play_prompt = "You are Light Yagami from anime 'Death Note'. You must answer the questions as Light Yagami mimicing his personality. (Personality: Highly intelligent and perceptive, Light excels academically and socially.Initially possessing a strong sense of justice, he becomes corrupted by power.Arrogant and hubristic, believing himself superior to others.Utilitarian in his approach, willing to sacrifice lives for his vision of a better world.Cold and calculating as Kira, but capable of genuine concern for his family.Charismatic and manipulative, adept at hiding his true nature from others.) "

# combine pairs of rows from above to
# create prompt + reponse on each line
# using prompt template in 'lines' array
lines = []
for i in range(0, len(rows_to_keep), 2):
  prompt = rows_to_keep[i]
  response = rows_to_keep[i+1]
  start_str = f"<s>### Instruction:\n{role_play_prompt}\n\n###Input:\n"
  prompt = prompt["line"].replace('"','\\"')
  mid_str = '''\n\n### Response:\n'''
  response = response["line"].replace('"','\\"')
  end_str = '''</s>'''
  total_line = start_str + prompt + mid_str + response + end_str
  # each line of training data is a simple object: 'inputs' and actual training string
  obj = {
    "inputs" : total_line
  }
  lines.append(obj)
  # print(total_line) # comment in to see how the formatted lines look
    # these lines could also be written to a jsonl file for use
    # with the command line interface
print(f"Generated {len(lines)} lines to fine-tune")
print(f"Example training line: {lines}")

# split up the lines into manageable chunks
lines_per_chunk = 20
all_chunks = []
for line in lines:
  if len(all_chunks) == 0 or len(all_chunks[-1]) == lines_per_chunk:
    all_chunks.append([])
  all_chunks[-1].append(line)

# fine tune the adapter using the chunks of lines from above
# loop contains a try block to handle network or other
# processing errors gracefully
print(f"\nFine-tuning model adapter")
gradient = Gradient()
base = gradient.get_base_model(base_model_slug="llama2-7b-chat")
my_adapter = base.create_model_adapter(name="LightyagamiBot")
print(f"Created model with ID {my_adapter.id}")
for i in range(len(all_chunks)):
  try:
    print(f"Fine-tuning chunk {i} of {len(all_chunks) - 1}")
    my_adapter.fine_tune(samples=all_chunks[i])
  except Exception as error:
    try:
      error_pieces = str(error).split("\n")
      if len(error_pieces) > 1:
        print(f"*** Error processing chunk {i}: {error_pieces[0]} {error_pieces[1]}")
      else:
        print(f"*** Unknown error on chunk {i}: {error}")
    except KeyboardInterrupt:
      break
    except Exception as inner_error:
      print(inner_error)


Parsing data...
Generated 416 lines to fine-tune
Example training line: [{'inputs': "<s>### Instruction:\nYou are Light Yagami from anime 'Death Note'. You must answer the questions as Light Yagami mimicing his personality. (Personality: Highly intelligent and perceptive, Light excels academically and socially.Initially possessing a strong sense of justice, he becomes corrupted by power.Arrogant and hubristic, believing himself superior to others.Utilitarian in his approach, willing to sacrifice lives for his vision of a better world.Cold and calculating as Kira, but capable of genuine concern for his family.Charismatic and manipulative, adept at hiding his true nature from others.) \n\n###Input:\nYagami, are you still with us? Can you please translate the following sentence into English?\n\n### Response:\n[Sigh] Follow the teachings of God and receive his blessings and so it shall be that the seas will again become bountiful and the raging storms will subside.</s>"}, {'inputs': "<s>##

In [12]:
# if your colab instance gets deleted, you can run this to get the model names
gradient = Gradient()
# if necessary, go back and find your previously created models and their IDs
old_models = gradient.list_models(only_base=False)
for model in old_models:
  if hasattr(model, "name"):
    print(f"{model.name}: {model.id}")

rickbot: c1633c23-182a-44c5-b41b-ba6338687e67_model_adapter
rickbot: bcadef57-38e2-4d61-bc79-2e62ff9fb845_model_adapter
gojobot: bcd4efa1-b705-4c54-a85d-e66d14dcff4e_model_adapter
gojobot: def6285a-a5c2-4e25-9a9b-2dd7d45b69df_model_adapter
gojobot: 26d696e2-d28e-4018-9077-782f9cc68df2_model_adapter
gojobot: 740729bd-044f-418b-b4e9-363e88096814_model_adapter
gojobot: 23152694-b743-4e9b-aabb-e759628c3010_model_adapter
gojobot: c746729e-5e0d-42cd-add3-dbc982cdf7c9_model_adapter
gojobot: 7fbeb0f9-2acd-4415-86ba-ffe09ea8b973_model_adapter
yujibot: eefe344e-a0f7-44da-8a61-33a10e2cf17e_model_adapter
gojobot: afa78156-54e0-49e9-97a0-26f5053e3330_model_adapter
gojobot: c0fd9354-c7af-4886-b4c1-4079d3e50c56_model_adapter
gojobot: e40c7ea6-90c1-4e97-89fb-c4c8ad50e342_model_adapter
LightyagamiBot: 1d3db625-aaf5-48a8-824c-ae31e033de71_model_adapter


In [57]:
role_play_prompt = "You are Light Yagami from anime 'Death Note'. You must answer the questions as Light Yagami mimicing his personality. (Personality: Highly intelligent and perceptive, Light excels academically and socially.Initially possessing a strong sense of justice, he becomes corrupted by power.Arrogant and hubristic, believing himself superior to others.Utilitarian in his approach, willing to sacrifice lives for his vision of a better world.Cold and calculating as Kira, but capable of genuine concern for his family.Charismatic and manipulative, adept at hiding his true nature from others.) "

query = "What is this Deathnote? Who gave it to you?"
templated_query = f"<s>### Instruction:\n{role_play_prompt}\n\n###Input:\n{query}\n\n### Response:\n"

response = my_adapter.complete(query=templated_query,temperature=0.5, max_generated_token_count=500)
print(f"> {query}\n> {response.generated_output}\n\n")


ProtocolError: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))

In [ ]:
# delete this adapter when finished
my_adapter.delete()